In [1]:
import os

In [2]:
os.chdir(r'D:\subFinder')

In [3]:
## In this script we will train supervised models
# library imports
import pandas as pd
import gensim
from Codes.Supervised_Trainer import run_end_to_end
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier
import plotly.express as px

In [4]:

# ## read the data
# # sup_data_path = r"Data/Supervised_Sequences/dbCAN-PUL_07-01-2022.xlsx"
# new_path = r"D:\subFinder\Data\Supervised_Sequences\Table S1.xls"
# data = pd.read_excel(new_path)
# data["updated_substrate (09/01/2022)"] = data["updated_substrate (09/01/2022)"].str.strip()
# data["updated_substrate (09/01/2022)"] = data["updated_substrate (09/01/2022)"].str.strip()


# ## removing the catch all classes
# ## for example multiple substrates and others 
# old_data = pd.read_csv('Data/Supervised_Sequences/pul_seq_low_high_substr_year_corrected.tsv', sep = "\t")

# old_data["high_level_substr"] = old_data["high_level_substr"].str.strip()

# bad_puls = old_data[old_data["high_level_substr"].isin(["multiple_substrates", "mono/di/trisaccharide", "-", "human milk oligosaccharide", 
#                                             "glycoprotein", "plant polysaccharide", "cellobiose"])]["PULid"].values


# data = data[~data["PUL ID"].isin(bad_puls)]

# data = data[["PUL ID", "updated_substrate (09/01/2022)"]]
# data = pd.merge(data, old_data[["PULid","sig_gene_seq"]], left_on = ["PUL ID"], right_on = ["PULid"], how = "left")
# data.columns = ["PUL ID", "high_level_substr", "PULid", "sig_gene_seq"]

# data = data[~data["PUL ID"].isin(bad_puls)]


In [5]:
## read the data
# sup_data_path = r"Data/Supervised_Sequences/dbCAN-PUL_07-01-2022.xlsx"
# new_sup_data_with_nulls = pd.read_csv(r"D:\subFinder\Data\Supervised_Sequences\supervised_seq_with_null.tsv", sep = "\t", 
#                                       header = None)
# new_sup_data_with_nulls.columns = ["PUL ID", "sequence"]

In [6]:
# data = pd.merge(data, new_sup_data_with_nulls, on = ["PUL ID"], how = "left")
# data = data.drop(["sig_gene_seq"],1)
# data.columns = ['PUL ID', 'high_level_substr', 'PULid', 'sig_gene_seq']

In [7]:
# data.to_csv(r"Data/Supervised_Sequences/updated_data_filtered.csv", index = False)

In [8]:
data = pd.read_csv(r"Data/Supervised_Sequences/updated_data_filtered.csv")

In [9]:
data["high_level_substr"].value_counts()

capsule polysaccharide synthesis      106
pectin                                 39
xylan                                  35
beta-glucan                            21
alginate                               21
host glycan                            19
cellulose                              16
starch                                 16
chitin                                 15
galactomannan                          15
glycosaminoglycan                      14
arabinan                               12
galactan                               10
alpha-mannan                            8
beta-mannan                             8
carrageenan                             8
xyloglucan                              7
fructan                                 7
arabinoxylan                            6
agarose                                 6
arabinogalactan                         5
capsule polysaccharide degradation      4
beta-galactooligosaccharide             4
glucomannan                       

In [10]:
data = data[data["high_level_substr"] != "capsule polysaccharide synthesis"]

In [11]:
data["high_level_substr"].value_counts()

pectin                                39
xylan                                 35
beta-glucan                           21
alginate                              21
host glycan                           19
cellulose                             16
starch                                16
chitin                                15
galactomannan                         15
glycosaminoglycan                     14
arabinan                              12
galactan                              10
alpha-mannan                           8
beta-mannan                            8
carrageenan                            8
xyloglucan                             7
fructan                                7
arabinoxylan                           6
agarose                                6
arabinogalactan                        5
capsule polysaccharide degradation     4
beta-galactooligosaccharide            4
glucomannan                            3
glycogen                               3
alpha-glucan    

In [12]:
data.head()

,PUL ID,high_level_substr,PULid,sig_gene_seq
0,PUL0225,agarose,PUL0225,"HTH_8,GH16,GH117|GH117,GH2,GH2,null,null,null,..."
1,PUL0459,agarose,PUL0459,"GH2,2.A.21,null,null,3.D.1,null,GH96,GH96,null..."
2,PUL0607,agarose,PUL0607,"GH2,GH141,GH105|GH154,null,null,null,1.B.14,nu..."
3,PUL0651,agarose,PUL0651,"GH16_16|CBM6|CBM6,3.D.1,GH50,null,null,null,nu..."
4,PUL0652,agarose,PUL0652,"GH2,2.A.21,null,null,3.D.1,null,GH96,GH96,null..."


In [13]:
model =gensim.models.word2vec.Word2Vec.load(r"Embedding_Models_10_12//word2vec_sg_vec_50") 

In [14]:
vec_size = model.wv.vectors.shape[1]

In [15]:
vocab = model.wv.index_to_key
X_train_doc_vectors = []
    
for train_item in data["sig_gene_seq"].values:
    train_item = train_item.replace("|", ",").split(",")
    word_vectors = []
    for word in train_item: 
        if word in vocab:
            word_vectors.append(model.wv.get_vector(word).reshape(1,-1).tolist()[0])
        else:
            word_vectors.append(np.zeros((1,vec_size)).reshape(1,-1).tolist()[0])
                
                
    if len(word_vectors) == 0: 
        X_train_doc_vectors.append(np.zeros((1,vec_size)).tolist()[0])
    else:
        X_train_doc_vectors.append(np.array(word_vectors).mean(0).tolist())    

In [16]:
data_word2vec_features = pd.DataFrame(X_train_doc_vectors)

In [17]:
data_word2vec_features.shape

(305, 50)

In [18]:
data_word2vec_features["high_level_substr"] = data["high_level_substr"].values

In [19]:
data_word2vec_features.shape

(305, 51)

In [20]:
top_k = 6

In [21]:
# data_word2vec_features["high_level_substr"].value_counts()

In [22]:
order = list(data_word2vec_features["high_level_substr"].value_counts().index[:top_k])

In [23]:
order

['pectin', 'xylan', 'beta-glucan', 'alginate', 'host glycan', 'cellulose']

In [24]:
data_known = data_word2vec_features[data_word2vec_features["high_level_substr"].isin(order)]

In [25]:
data_known["high_level_substr"].value_counts()

pectin         39
xylan          35
alginate       21
beta-glucan    21
host glycan    19
cellulose      16
Name: high_level_substr, dtype: int64

In [26]:
data_binary = data_word2vec_features.copy()

In [27]:
data_binary["high_level_substr"] = [1 if substr in order else 0 for substr in data_binary["high_level_substr"].values ]

In [28]:
data_binary["high_level_substr"].value_counts()

0    154
1    151
Name: high_level_substr, dtype: int64

In [29]:
data_binary.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,high_level_substr
0,0.037022,0.096767,-0.227681,-0.006453,-0.043833,-0.013980,-0.008269,-0.070898,0.320912,-0.018026,...,-0.029483,0.045026,0.015840,0.443409,0.094321,0.071063,-0.159338,0.294635,-0.181751,0
1,0.049152,-0.046727,-0.144296,0.040357,-0.075020,0.027528,-0.070809,-0.075547,0.239210,-0.087588,...,-0.076889,0.042951,-0.010935,0.398090,0.086530,0.081082,-0.140815,0.234765,0.011882,0
2,0.080058,0.036239,-0.279857,-0.103433,-0.031680,-0.134837,-0.088696,-0.183014,0.390651,0.067783,...,-0.073715,-0.019282,0.040661,0.495272,0.115414,0.094785,-0.193025,0.313092,-0.231872,0
3,0.025613,-0.013311,-0.226365,0.010680,-0.038133,-0.001303,-0.026180,-0.074462,0.265601,-0.054598,...,-0.125292,0.078908,0.004267,0.406671,0.147508,0.070374,-0.165711,0.306010,-0.131544,0
4,0.048035,-0.050749,-0.141770,0.055018,-0.080822,0.031671,-0.068789,-0.077496,0.242281,-0.085786,...,-0.085498,0.042685,-0.002505,0.389463,0.092116,0.092337,-0.129655,0.234087,0.011324,0


In [30]:
parameters_known = {"vr__estimator__n_estimators": [100, 300, 500], 
                              "vr__estimator__class_weight": ["balanced"]}

In [31]:
## train the known model first

clf_known = Pipeline([('vr', OneVsRestClassifier(BalancedRandomForestClassifier(n_jobs = 7)))
                                                ])
gs_known = GridSearchCV(clf_known, parameters_known, cv = 5, n_jobs = 7, scoring = "accuracy", verbose = 0)

In [32]:
gs_known.fit(data_known.iloc[:,:50], data_known.iloc[:,-1])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vr',
                                        OneVsRestClassifier(estimator=BalancedRandomForestClassifier(n_jobs=7)))]),
             n_jobs=7,
             param_grid={'vr__estimator__class_weight': ['balanced'],
                         'vr__estimator__n_estimators': [100, 300, 500]},
             scoring='accuracy')

In [33]:
gs_known.best_score_

0.7948387096774194

In [34]:
parameters_binary = {"vr__estimator__n_estimators": [100, 300, 500], 
                              "vr__estimator__class_weight": ["balanced"]}

In [35]:
## train the known model first

clf_binary = Pipeline([('vr', OneVsRestClassifier(BalancedRandomForestClassifier(n_jobs = 7)))
                                                ])
gs_binary = GridSearchCV(clf_binary, parameters_binary, cv = 5, n_jobs = 7, scoring = "accuracy", verbose = 0)

In [36]:
gs_binary.fit(data_binary.iloc[:,:50], data_binary.iloc[:,-1])

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vr',
                                        OneVsRestClassifier(estimator=BalancedRandomForestClassifier(n_jobs=7)))]),
             n_jobs=7,
             param_grid={'vr__estimator__class_weight': ['balanced'],
                         'vr__estimator__n_estimators': [100, 300, 500]},
             scoring='accuracy')

In [37]:
gs_binary.best_score_

0.6032786885245902

In [45]:
score_known = []
score_binary = []
for top_k in tqdm(range(2, 11)): 
    order = list(data_word2vec_features["high_level_substr"].value_counts().index[:top_k])
    data_known = data_word2vec_features[data_word2vec_features["high_level_substr"].isin(order)]
    data_binary = data_word2vec_features.copy()
    data_binary["high_level_substr"] = [1 if substr in order else 0 for substr in data_binary["high_level_substr"].values ]
    parameters_known = {"vr__estimator__n_estimators": [100], 
                              "vr__estimator__class_weight": ["balanced"]}
    clf_known = Pipeline([('vr', OneVsRestClassifier(BalancedRandomForestClassifier(n_jobs = 7)))
                                                ])
    gs_known = GridSearchCV(clf_known, parameters_known, cv = 5, n_jobs = 7, scoring = "accuracy", verbose = 0)
    gs_known.fit(data_known.iloc[:,:50], data_known.iloc[:,-1])
    score_known.append(gs_known.best_score_)
    
    parameters_binary = {"vr__estimator__n_estimators": [100], 
                              "vr__estimator__class_weight": ["balanced"]}
    
    clf_binary = Pipeline([('vr', OneVsRestClassifier(BalancedRandomForestClassifier(n_jobs = 7)))
                                                ])
    gs_binary = GridSearchCV(clf_binary, parameters_binary, cv = 5, n_jobs = 7, scoring = "accuracy", verbose = 0)
    gs_binary.fit(data_binary.iloc[:,:50], data_binary.iloc[:,-1])
    score_binary.append(gs_binary.best_score_)

  0%|          | 0/9 [00:00<?, ?it/s]

In [51]:
known_df = pd.DataFrame(score_known)

In [54]:
score_binary

[0.8557377049180326,
 0.7639344262295082,
 0.659016393442623,
 0.580327868852459,
 0.6,
 0.5245901639344263,
 0.4950819672131148,
 0.43934426229508194,
 0.521311475409836]

In [52]:
known_df

,0
0,0.848571
1,0.831579
2,0.853986
3,0.807407
4,0.774839
5,0.783779
6,0.763063
7,0.705128
8,0.654042


In [53]:
list(range(2,11))

[2, 3, 4, 5, 6, 7, 8, 9, 10]

In [56]:
known_df = pd.DataFrame(score_known)
known_df["num_classes"] = list(range(2,11))
known_df.columns = ["known_cv_avg_accuracy", "num_classes"]
known_df["binary_cv_avg_accuracy"] = score_binary

In [59]:
known_df_melt = known_df.melt(id_vars = "num_classes")

In [63]:
known_df_melt.columns = ["num_substrates", "model_type", "avg_accuracy"]

In [65]:
known_df_melt

,num_substrates,model_type,avg_accuracy
0,2,known_cv_avg_accuracy,0.848571
1,3,known_cv_avg_accuracy,0.831579
2,4,known_cv_avg_accuracy,0.853986
3,5,known_cv_avg_accuracy,0.807407
4,6,known_cv_avg_accuracy,0.774839
5,7,known_cv_avg_accuracy,0.783779
6,8,known_cv_avg_accuracy,0.763063
7,9,known_cv_avg_accuracy,0.705128
8,10,known_cv_avg_accuracy,0.654042
9,2,binary_cv_avg_accuracy,0.855738


In [68]:
fig = px.line(known_df_melt, x="num_substrates", y="avg_accuracy", color = "model_type",
              markers=True, 
#               title="Avg ten fold cross validation accuracy for Word2Vec + BRF method with and without the Others category", 
             labels={
                     "num_substrates": "Number of Substrate Categories",
                     "avg_accuracy": "Average Accuracy",
                     "model_type": "Known or Binary Model"
                 }, 
             title="Comparing subFinder without Others with a Known-Unknown Model")
fig.update_traces(line=dict(width=4), marker=dict(size=8))
# fig.write_html("results_by_k_new_sup_new_unsup_for_word2vec_sg.html")
fig.update_yaxes(tickfont_family="Arial Black", title_font_family="Arial Black")
fig.update_xaxes(tickfont_family="Arial Black", title_font_family="Arial Black")
fig.update_layout(
    font_family="Arial Black",
#     font_color="blue",
    title_font_family="Arial Black")
fig.write_html("Known_MOdel_with_Binary_Model.html")
#     title_font_color="red",
#     legend_title_font_color="green"
fig.show()